In [339]:
best_Action = []

## 강화학습 모델 생성

In [618]:
from scipy.io import loadmat
import numpy as np
np.set_printoptions(formatter={'float': '{:0.5f}'.format})

import pandas as pd

# .mat 파일 로드
data = loadmat('pvLoadPriceData_test.mat')

# 데이터 추출 예시
# MATLAB 파일 내에 있는 변수 이름을 정확히 알아야 합니다. time, cloudyDay, clearDay, loadData, costData
time = data['time'].flatten()
solarData = data['clearDay'].flatten() * 10e3  # [Kw -> W]
loadData = data['loadData'][:, 2].flatten()    # [W]
costData = data['costData'].flatten() 

print("자료형:", type(time), "차원" , time.shape, )
print("자료형:", type(solarData), "차원" , solarData.shape, )
print("자료형:", type(loadData), "차원" , loadData.shape, )
print("자료형:", type(costData), "차원" , costData.shape, )

solarData = solarData.tolist()
loadData = loadData.tolist()
costData = costData.tolist()

자료형: <class 'numpy.ndarray'> 차원 (228,)
자료형: <class 'numpy.ndarray'> 차원 (228,)
자료형: <class 'numpy.ndarray'> 차원 (228,)
자료형: <class 'numpy.ndarray'> 차원 (228,)


In [593]:
def W2KW(W) :
    Kw = W * 10e-4
    return Kw

print(W2KW(1000))

1.0


In [619]:
# 전력 -> 수소
def P2G(power_kW, time_hours=1):
    """
    Calculate the amount of hydrogen produced using PEM electrolysis.

    Parameters:
    power_kW (float): The power used in kilowatts.
    efficiency (float): The efficiency of the electrolysis process (0-1).
    time_hours (float): The duration for which the power is applied in hours.

    Returns:
    float: The amount of hydrogen produced in kilograms.
    """
    efficiency = 0.7  # Efficiency of the process
    heating_value_hydrogen = 33.33  # Heating value of hydrogen in kWh/kg

    hydrogen_produced_kg = (power_kW * time_hours * efficiency * heating_value_hydrogen) / 1000
    return hydrogen_produced_kg

# Example usage:
power = 100  # Power in kW

# Calculate hydrogen production for 1 hour
hydrogen_output = P2G(power)
print(f"Produced hydrogen: {hydrogen_output:.5f} kg")

Produced hydrogen: 2.33310 kg


In [620]:
# 남은 수소를 다시 전력으로
def P2Ga(hydrogen_produced_kg, time_hours=1):
    """
    Calculate the amount of hydrogen produced using PEM electrolysis.

    Parameters:
    power_kW (float): The power used in kilowatts.
    efficiency (float): The efficiency of the electrolysis process (0-1).
    time_hours (float): The duration for which the power is applied in hours.

    Returns:
    float: The amount of hydrogen produced in kilograms.
    """
    efficiency = 0.7  # Efficiency of the process
    heating_value_hydrogen = 33.33  # Heating value of hydrogen in kWh/kg

    power_kW = (hydrogen_produced_kg * 1000) / (time_hours * efficiency * heating_value_hydrogen)
    return power_kW

# Example usage:
power = 2.33310  # Power in kW

# Calculate hydrogen production for 1 hour
hydrogen_output = P2Ga(power)
print(f"Produced hydrogen: {hydrogen_output:.5f} kWh")

Produced hydrogen: 100.00000 kWh


In [621]:
# 수소 -> 전력
def G2P(hydrogen_kg):
    """
    Calculate the amount of electricity produced from hydrogen using a PEM fuel cell.

    Parameters:
    hydrogen_kg (float): The amount of hydrogen used in kilograms.
    fuel_cell_efficiency (float): The efficiency of the fuel cell (0-1).

    Returns:
    float: The amount of electricity produced in kilowatt-hours.
    """
    efficiency = 0.5  # Efficiency of the fuel cell

    heating_value_hydrogen = 33.33  # Heating value of hydrogen in kWh/kg
    electricity_produced_kWh = heating_value_hydrogen * efficiency * hydrogen_kg
    return electricity_produced_kWh

# Example usage:
hydrogen_amount = 2.33310  # Amount of hydrogen in kg
# Calculate electricity production
electricity_output = G2P(hydrogen_amount)
print(f"Electricity produced: {electricity_output:.5f} kWh")

Electricity produced: 38.88111 kWh


In [622]:
# 가스를 수소로 바꿨을 때 x만큼 필요함. 얼만큼의 수소를 쓸지 계산할 때
def G2Pa(tNeedPower):
    """
    Calculate the amount of electricity produced from hydrogen using a PEM fuel cell.

    Parameters:
    hydrogen_kg (float): The amount of hydrogen used in kilograms.
    fuel_cell_efficiency (float): The efficiency of the fuel cell (0-1).

    Returns:
    float: The amount of electricity produced in kilowatt-hours.
    """
    efficiency = 0.5  # Efficiency of the fuel cell

    heating_value_hydrogen = 33.33  # Heating value of hydrogen in kWh/kg
    tNeedHydrogen = tNeedPower / (heating_value_hydrogen * efficiency) 
    return tNeedHydrogen

# Example usage:
hydrogen_amount = 19321.93141 * 10e-3 # Amount of hydrogen in kg
# Calculate electricity production
electricity_output = G2Pa(hydrogen_amount)
print(f"Electricity produced: {electricity_output:.5f} Kg")

Electricity produced: 11.59432 Kg


In [653]:
import numpy as np
import gym
from gym import spaces

# 전력망 환경 클래스 정의
class PowerGridEnv(gym.Env):
    def __init__(self):
        self.solar_data = solarData
        self.load_data = loadData
        self.price_data = costData

        self.HydroCap = 5000  # 수소 저장 장치 용량 [kg]
        HydrogenEnergyDensity = 33.33 # 수소 1Kg 당 [kWh]
        self.HydrogenEnergy = self.HydroCap * HydrogenEnergyDensity # 총 에너지 용량
        
        self.Hydrogen = 0.5 * self.HydroCap
        self.HydrogenMin = 0.2 * self.HydroCap
        self.HydrogenMax = 0.9 * self.HydroCap

        self.total_grid_cost = 0
        self.state = None

        # 배터리 상태
        self.observation_space = spaces.Box(low=np.array([0]), high=np.array([100]))
        
        # 액션 종류
        self.action_space = spaces.Discrete(7)
        self.data_pointer = 0
        

    def reset(self):
        self.data_pointer = 0
        self.Hydrogen = 0.5 * self.HydroCap  # 수소 가스 상태
        self.total_grid_cost = 0
        action = 0
        reward = 0
        done = False

        tP2G = 0
        tG2P = 0
        tNeedPower = 0

        self.state = [self.solar_data[0], self.load_data[0], (self.Hydrogen/self.HydroCap) * 100, (tP2G/self.HydroCap) * 100, ((-tG2P/self.HydroCap) * 100), action, self.total_grid_cost* 10e-6]
        return np.array(self.state), reward, done, {"cost" : self.total_grid_cost}

    
    def render(self, mode='human'):
        # 가정: self.state는 환경의 현재 상태를 나타냅니다.
        # 간단한 텍스트 출력으로 상태를 표현합니다.
        print(f"Current state: {self.state}")
        
    def step(self, action):
        reward = 0
       
        tPV = self.solar_data[self.data_pointer]    # 스탭 당 태양광 발전량
        tLoad = self.load_data[self.data_pointer]   # 스탭 당 부하량
        tPrice = self.price_data[self.data_pointer] # 스탭 당 전력 가격

        tP2G = 0
        tG2P = 0
        tNeedPower = 0

        # 액션에 따른 배터리 및 그리드 상호작용
        if action == 0 : # 태양광 O 수소 X 그리드 X
            if tPV >= tLoad :
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 수소로 변환 후 저장
                if tSurplusPower > 0 : 
                    """
                    1. 충전 가능량 확인하기
                    2. 충전 가능량 만큼만 잉여 전력을 수소 가스로 변환하기
                    3. 남은 잉여 전력은 그리드 판매
                    """
                    # 수소 탱크의 충전 가능(kg)
                    tPossible_Hydrogen = self.HydrogenMax - self.Hydrogen

                    # Power[kW] to Gas[kg]
                    tP2G = P2G(tSurplusPower) # 잉여 전력 -> 수소 가스로 변환
                    
                    

                    # 수소 탱크에 충전할 전력 계산
                    tPowerChg = min(tP2G, tPossible_Hydrogen) # 잉여 수소 가스랑 수소 탱크 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2G - tPowerChg # 충전하고 남은 전력량
                    tP2G = tPowerChg
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.Hydrogen + tPowerChg) > self.HydrogenMin) and ((self.Hydrogen + tPowerChg) < self.HydrogenMax) :
                        self.Hydrogen += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.Hydrogen += tPowerChg
                        self.total_grid_cost -= P2Ga(tSurplusChg) * tPrice
                        reward = -P2Ga(tSurplusChg) * tPrice

            else : # tLoad > tPV
                # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                tNeedPower = tLoad - tPV
                tG2P = G2Pa(tNeedPower) # 필요한 수소 가스 계산[kg]

                # 수소 탱크의 방전 가능(%)
                tPossible_Hydrogen = self.Hydrogen - self.HydrogenMin

                # 수소 텡크에서 사용할 양(L) 계산
                tPowerDisChg = min(tG2P, tPossible_Hydrogen) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tG2P - tPowerDisChg # 충전하고 남은 전력량
                tG2P = tPowerDisChg
                
                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.Hydrogen - tPowerDisChg) > self.HydrogenMin) and ((self.Hydrogen - tPowerDisChg) < self.HydrogenMax) :
                    self.Hydrogen -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.Hydrogen -= tPowerDisChg
                    self.total_grid_cost += tNeedPower * tPrice
                    reward = tNeedPower * tPrice


        elif action == 1 : # 태양광 O 수소 O 그리드 X
            if tLoad > tPV :
                # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                tNeedPower = tLoad - tPV
                tG2P = G2Pa(tNeedPower) # 필요한 수소 가스 계산[kg]

                # 수소 탱크의 방전 가능(%)
                tPossible_Hydrogen = self.Hydrogen - self.HydrogenMin

                # 수소 텡크에서 사용할 양(L) 계산
                tPowerDisChg = min(tG2P, tPossible_Hydrogen) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tG2P - tPowerDisChg # 방전하고 남은 전력량
                tG2P = tPowerDisChg

                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.Hydrogen - tPowerDisChg) > self.HydrogenMin) and ((self.Hydrogen - tPowerDisChg) < self.HydrogenMax) :
                    self.Hydrogen -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.Hydrogen -= tPowerDisChg
                    self.total_grid_cost += tNeedPower * tPrice
                    reward = tNeedPower * tPrice


            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 수소로 변환 후 저장
                if tSurplusPower > 0 : 
                    # Power[kW] to Gas[kg]
                    tP2G = P2G(tSurplusPower) # 잉여 전력 -> 수소 가스로 변환
                    
                    # 잉여 전력량
                    # tP2B = (tSurplusPower / self.battEnergy) * 100

                    # 수소 탱크의 충전 가능(kg)
                    tPossible_Hydrogen = self.HydrogenMax - self.Hydrogen

                    # 수소 탱크에 충전할 전력 계산
                    tPowerChg = min(tP2G, tPossible_Hydrogen) # 잉여 수소 가스랑 수소 탱크 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2G - tPowerChg # 충전하고 남은 전력량
                    tP2G = tPowerChg
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.Hydrogen + tPowerChg) > self.HydrogenMin) and ((self.Hydrogen + tPowerChg) < self.HydrogenMax) :
                        self.Hydrogen += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.Hydrogen += tPowerChg
                        self.total_grid_cost -= P2Ga(tSurplusChg) * tPrice
                        reward = -P2Ga(tSurplusChg) * tPrice

    
        elif action == 2 : # 태양광 O 수소 X 그리드 O
            if tLoad > tPV :
                tNeedPower = tLoad - tPV # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                if tNeedPower > 0 : # 전력이 더 필요할 경우
                    self.total_grid_cost += (tNeedPower * tPrice)
                    reward = (tNeedPower * tPrice)

            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 수소로 변환 후 저장
                if tSurplusPower > 0 : 
                    # Power[kW] to Gas[kg]
                    tP2G = P2G(tSurplusPower) # 잉여 전력 -> 수소 가스로 변환
                    
                    # 수소 탱크의 충전 가능(kg)
                    tPossible_Hydrogen = self.HydrogenMax - self.Hydrogen

                    # 수소 탱크에 충전할 전력 계산
                    tPowerChg = min(tP2G, tPossible_Hydrogen) # 잉여 수소 가스랑 수소 탱크 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2G - tPowerChg # 충전하고 남은 전력량
                    tP2G = tPowerChg
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.Hydrogen + tPowerChg) > self.HydrogenMin) and ((self.Hydrogen + tPowerChg) < self.HydrogenMax) :
                        self.Hydrogen += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.Hydrogen += tPowerChg
                        self.total_grid_cost -= P2Ga(tSurplusChg) * tPrice
                        reward = -P2Ga(tSurplusChg) * tPrice


        elif action == 3 : # 태양광 O 수소 O 그리드 O
            if tLoad > tPV :
                # 태양광 발전량으로 커버한 뒤 필요한 전력을 계산
                tNeedPower = tLoad - tPV
                tG2P = G2Pa(tNeedPower) # 필요한 수소 가스 계산[kg]

                # 수소 탱크의 방전 가능(%)
                tPossible_Hydrogen = self.Hydrogen - self.HydrogenMin

                # 수소 텡크에서 사용할 양(L) 계산
                tPowerDisChg = min(tG2P, tPossible_Hydrogen) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tG2P - tPowerDisChg # 충전하고 남은 전력량
                tG2P = tPowerDisChg
                
                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.Hydrogen - tPowerDisChg) > self.HydrogenMin) and ((self.Hydrogen - tPowerDisChg) < self.HydrogenMax) :
                    self.Hydrogen -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.Hydrogen -= tPowerDisChg
                    self.total_grid_cost += tNeedPower * tPrice
                    reward = tNeedPower * tPrice
             

            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 수소로 변환 후 저장
                if tSurplusPower > 0 : 
                    # Power[kW] to Gas[kg]
                    tP2G = P2G(tSurplusPower) # 잉여 전력 -> 수소 가스로 변환
                    
                    # 수소 탱크의 충전 가능(kg)
                    tPossible_Hydrogen = self.HydrogenMax - self.Hydrogen

                    # 수소 탱크에 충전할 전력 계산
                    tPowerChg = min(tP2G, tPossible_Hydrogen) # 잉여 수소 가스랑 수소 탱크 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2G - tPowerChg # 충전하고 남은 전력량
                    tP2G = tPowerChg
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.Hydrogen + tPowerChg) > self.HydrogenMin) and ((self.Hydrogen + tPowerChg) < self.HydrogenMax) :
                        self.Hydrogen += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.Hydrogen += tPowerChg
                        self.total_grid_cost -= P2Ga(tSurplusChg) * tPrice
                        reward = -P2Ga(tSurplusChg) * tPrice
                            

        elif action == 4 : # 태양광 X 수소 O 그리드 X
            if tLoad > tPV :
                # 필요한 수소 가스 계산[kg]
                tG2P = G2Pa(tLoad)
                # 수소 탱크의 방전 가능(%)
                tPossible_Hydrogen = self.Hydrogen - self.HydrogenMin

                # 수소 텡크에서 사용할 양(L) 계산
                tPowerDisChg = min(tG2P, tPossible_Hydrogen) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tG2P - tPowerDisChg # 충전하고 남은 전력량
                tG2P = tPowerDisChg

                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.Hydrogen - tPowerDisChg) > self.HydrogenMin) and ((self.Hydrogen - tPowerDisChg) < self.HydrogenMax) :
                    self.Hydrogen -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.Hydrogen -= tPowerDisChg
                    self.total_grid_cost += tNeedPower * tPrice
                    reward = tNeedPower * tPrice
            
            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 수소로 변환 후 저장
                if tSurplusPower > 0 : 
                    # Power[kW] to Gas[kg]
                    tP2G = P2G(tSurplusPower) # 잉여 전력 -> 수소 가스로 변환
                    
                    # 수소 탱크의 충전 가능(kg)
                    tPossible_Hydrogen = self.HydrogenMax - self.Hydrogen

                    # 수소 탱크에 충전할 전력 계산
                    tPowerChg = min(tP2G, tPossible_Hydrogen) # 잉여 수소 가스랑 수소 탱크 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2G - tPowerChg # 충전하고 남은 전력량
                    tP2G = tPowerChg
                    
                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.Hydrogen + tPowerChg) > self.HydrogenMin) and ((self.Hydrogen + tPowerChg) < self.HydrogenMax) :
                        self.Hydrogen += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.Hydrogen += tPowerChg
                        self.total_grid_cost -= P2Ga(tSurplusChg) * tPrice 
                        reward = -P2Ga(tSurplusChg) * tPrice 


        elif action == 5 : # 태양광 X 수소 X 그리드 O
            self.total_grid_cost += tLoad * tPrice
            reward = tLoad * tPrice


        elif action == 6 : # 태양광 X 수소 O 그리드 O
            if tLoad > tPV :
                # 필요한 수소 가스 계산[kg]
                tG2P = G2Pa(tLoad)
                # 수소 탱크의 방전 가능(%)
                tPossible_Hydrogen = self.Hydrogen - self.HydrogenMin

                # 수소 텡크에서 사용할 양(L) 계산
                tPowerDisChg = min(tG2P, tPossible_Hydrogen) # 필요한 전력이랑 배터리 뱡젼 가능량을 비교해서 더 작은 값을 방전
                tSurplusChg = tG2P - tPowerDisChg # 충전하고 남은 전력량
                tG2P = tPowerDisChg

                # 배터리를 방전했을 때 20~80% 일 때
                if ((self.Hydrogen - tPowerDisChg) > self.HydrogenMin) and ((self.Hydrogen - tPowerDisChg) < self.HydrogenMax) :
                    self.Hydrogen -= tPowerDisChg
                # 배터리를 방전했을 때 20% 이하일 때
                else :
                    self.Hydrogen -= tPowerDisChg
                    self.total_grid_cost += tNeedPower * tPrice
                    reward = tNeedPower * tPrice
            
            else : # tLoad < tPV 
                tSurplusPower = tPV - tLoad # 잉여 전력 계산 -> 잉여 전력이 있다면 수소로 변환 후 저장
                if tSurplusPower > 0 : 
                    # Power[kW] to Gas[kg]
                    tP2G = P2G(tSurplusPower) # 잉여 전력 -> 수소 가스로 변환
                    
                    # 수소 탱크의 충전 가능(kg)
                    tPossible_Hydrogen = self.HydrogenMax - self.Hydrogen

                    # 수소 탱크에 충전할 전력 계산
                    tPowerChg = min(tP2G, tPossible_Hydrogen) # 잉여 수소 가스랑 수소 탱크 충전 가능량을 비교해서 더 작은 값을 충전
                    tSurplusChg = tP2G - tPowerChg # 충전하고 남은 전력량
                    tP2G = tPowerChg

                    # 배터리를 충전했을 때 20~80% 일 때
                    if ((self.Hydrogen + tPowerChg) > self.HydrogenMin) and ((self.Hydrogen + tPowerChg) < self.HydrogenMax) :
                        self.Hydrogen += tPowerChg
                    # 배터리를 충전했을 때 80% 이상일 때
                    else :
                        self.Hydrogen += tPowerChg
                        self.total_grid_cost -= P2Ga(tSurplusChg) * tPrice 
                        reward = -P2Ga(tSurplusChg) * tPrice 


        # 데이터 포인터 갱신
        if self.data_pointer >= len(self.solar_data) - 1:
            done = True
        else:
            done = False
            self.data_pointer += 1

        # reward = 1 if done else 0
        self.state = [tPV, tLoad, (self.Hydrogen/self.HydroCap) * 100, (tP2G/self.HydroCap) * 100, ((-tG2P/self.HydroCap) * 100), action, self.total_grid_cost* 10e-6]
        return np.array(self.state), reward * 10e-6, done, {"cost" : self.total_grid_cost}


# 환경 인스턴스 생성 및 초기화
env = PowerGridEnv()
state = env.reset()
done = False

print("       PV        Load      SOC(%)   충전량   방전량     액션   누적 비용")

while not done:
    action = env.action_space.sample()  # 무작위 액션
    state, reward, done, info = env.step(action)
    
    print("", state, "보상", reward)


       PV        Load      SOC(%)   충전량   방전량     액션   누적 비용
 [35000.00000 5215.40860 63.89809 13.89809 0.00000 2.00000 0.00000] 보상 0.0
 [35100.00000 13735.69150 73.86710 9.96901 0.00000 6.00000 0.00000] 보상 0.0
 [34900.00000 22835.42484 79.49667 5.62957 0.00000 4.00000 0.00000] 보상 0.0
 [34500.00000 42857.48440 79.49667 0.00000 0.00000 2.00000 6.39348] 보상 6.393475562262794
 [34500.00000 53821.93141 20.00000 0.00000 -59.49667 4.00000 6.39348] 보상 0.0
 [34700.00000 44613.88665 20.00000 0.00000 0.00000 2.00000 13.97760] 보상 7.58412328864647
 [34700.00000 48866.83350 20.00000 0.00000 0.00000 5.00000 51.36073] 보상 37.38312762779666
 [34900.00000 44613.88665 20.00000 0.00000 -0.00000 0.00000 58.79185] 보상 7.43112328864647
 [34500.00000 48866.83350 20.00000 0.00000 0.00000 5.00000 96.17498] 보상 37.38312762779666
 [34500.00000 44613.88665 20.00000 0.00000 -0.00000 4.00000 96.17498] 보상 0.0
 [34700.00000 48866.83350 20.00000 0.00000 -0.00000 4.00000 96.17498] 보상 0.0
 [34700.00000 34917.21196 20.00000 

In [624]:
env.reset()


(array([35000.00000, 5215.40860, 50.00000, 0.00000, 0.00000, 0.00000,
        0.00000]),
 0,
 False,
 {'cost': 0})

In [642]:
import random
import matplotlib.pyplot as plt

# 환경 초기화
env = PowerGridEnv()
state_size = len(loadData)
action_size = env.action_space.n

q_table = np.zeros((state_size, action_size))
alpha = 0.1     # 학습률
gamma = 0.9    # 할인율
epsilon = 0.1   # 탐험률

# 학습 과정
for i in range(1000):
    num = 0
    state = env.reset()
    done = False
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmin(q_table[num])

        next_state, reward, done, _ = env.step(action)
        old_value = q_table[num, action]
        next_max = np.max(q_table[num])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[num, action] = new_value
        num += 1


In [644]:
q_table

array([[-17.17628, -13.90392, -16.72465, -11.36149, -12.48881, 5.58337,
        -12.54773],
       [-7.68592, -6.16076, -7.70641, -7.70383, -7.70634, 10.04734,
        -6.43887],
       [11.44335, 11.29567, 11.30517, 11.44335, 11.29567, 24.44655,
        11.29623],
       [50.56376, 50.15297, 50.11246, 50.02440, 50.10712, 57.12225,
        50.11660],
       [44.89944, 44.73136, 44.63916, 45.89327, 44.62228, 51.30314,
        44.46292],
       [42.51763, 42.88808, 43.64976, 42.73550, 42.10193, 48.07045,
        42.10295],
       [43.48970, 43.56809, 42.44701, 42.43847, 42.13451, 49.62904,
        41.85819],
       [48.53078, 46.01503, 47.07203, 46.67455, 45.94448, 53.97669,
        45.94207],
       [42.03164, 42.62238, 41.44753, 41.09723, 40.92688, 46.56260,
        40.92688],
       [53.71486, 53.82635, 52.85200, 52.74419, 52.78673, 62.36669,
        52.78673],
       [36.47601, 38.79918, 38.24726, 36.42145, 35.53724, 39.49432,
        35.53799],
       [27.07599, 27.07599, 27.08744, 

In [652]:
np.argmin(q_table[1])

2

In [655]:
done = False
num = 0
env.reset()
best_action_sequence = []  # 최적 액션 시퀀스 저장

while not done:
    action = np.argmin(q_table[num])
    best_action_sequence.append(action)  # 액션 시퀀스에 추가

    state, reward, done, _ = env.step(action)
    num += 1
    env.render()

Current state: [35000.0, 5215.408596026129, 63.89808604092229, 13.898086040922287, 0.0, 0, 0.0]
Current state: [35100.0, 13735.69150206051, 73.86709967223082, 9.969013631308524, 0.0, 2, 0.0]
Current state: [34900.0, 22835.42483980296, 79.49667173348196, 5.629572061251142, 0.0, 1, 0.0]
Current state: [34500.0, 42857.48439511476, 69.46668746091699, 0.0, -10.029984272564974, 3, 0.0]
Current state: [34500.0, 53821.9314050352, 20.0, 0.0, -49.466687460916994, 6, 0.0]
Current state: [34700.0, 44613.88665182545, 20.0, 0.0, -0.0, 4, 0.0]
Current state: [34700.0, 48866.83350038779, 20.0, 0.0, -0.0, 6, 0.0]
Current state: [34900.0, 44613.88665182545, 20.0, 0.0, -0.0, 6, 0.0]
Current state: [34500.0, 48866.83350038779, 20.0, 0.0, -0.0, 4, 0.0]
Current state: [34500.0, 44613.88665182545, 20.0, 0.0, -0.0, 3, 7.73712328864647]
Current state: [34700.0, 48866.83350038779, 20.0, 0.0, -0.0, 4, 7.73712328864647]
Current state: [34700.0, 34917.21196014155, 20.0, 0.0, -0.0, 3, 7.9032904381547535]
Current st

In [651]:
done = False
env.reset()
i = 0
print("       PV        Load      SOC(%)   충전량   방전량     액션   누적 비용")

while not done:
    state, reward, done, _ = env.step(best_action_sequence[i])
    i += 1
    print("", state, "보상", reward)


       PV        Load      SOC(%)   충전량   방전량     액션   누적 비용
 [35000.00000 5215.40860 90.00000 40.00000 0.00000 0.00000 -22.12943] 보상 -22.129432560339357
 [35100.00000 13735.69150 90.00000 0.00000 0.00000 2.00000 -38.47313] 보상 -16.34369600092371
 [34900.00000 22835.42484 90.00000 0.00000 0.00000 1.00000 -47.70253] 보상 -9.229399997550736
 [34500.00000 42857.48440 20.00000 0.00000 -70.00000 3.00000 -41.30905] 보상 6.393475562262794
 [34500.00000 53821.93141 20.00000 0.00000 -0.00000 6.00000 -41.30905] 보상 0.0
 [34700.00000 44613.88665 20.00000 0.00000 -0.00000 4.00000 -41.30905] 보상 0.0
 [34700.00000 48866.83350 20.00000 0.00000 -0.00000 6.00000 -41.30905] 보상 0.0
 [34900.00000 44613.88665 20.00000 0.00000 -0.00000 6.00000 -41.30905] 보상 0.0
 [34500.00000 48866.83350 20.00000 0.00000 -0.00000 4.00000 -41.30905] 보상 0.0
 [34500.00000 44613.88665 20.00000 0.00000 -0.00000 3.00000 -33.57193] 보상 7.73712328864647
 [34700.00000 48866.83350 20.00000 0.00000 -0.00000 4.00000 -33.57193] 보상 0.0
 [34700.00

In [629]:
# 환경 생성
env = PowerGridEnv()

# 시뮬레이션할 에피소드 수 설정
episodes = 1000000

min_cost = float('inf')  # 최소 비용을 기록하기 위한 변수
best_state = None  # 최적의 상태를 저장하기 위한 변수
best_action_sequence = []  # 최적 액션 시퀀스 저장

# 에피소드 반복 실행
for episode in range(1, episodes + 1):
    state = env.reset()  # 환경 초기화
    done = False 
    total_reward = 0
    action_sequence = []  # 현재 에피소드의 액션 시퀀스

    while not done:
        action = env.action_space.sample()  # 무작위 액션 선택
        action_sequence.append(action)  # 액션 시퀀스에 추가
        next_state, reward, done, info = env.step(action)  # 액션 적용    
        total_reward += reward

        # 비용 최소화 조건 검사
        if done and info['cost'] < min_cost:
            min_cost = info['cost']
            best_state = state  # 최적 상태 갱신
            best_action_sequence = action_sequence  # 최적 액션 시퀀스 갱신

    # 에피소드 결과 출력
    if episode % 1000 == 0:  # 진행 상황을 1000 에피소드마다 출력
        print(f'Episode: {episode}, Total Reward: {total_reward}, Minimum Cost: { min_cost * 10e-6}')
    

# 최적 상태와 그때의 비용 출력
print(f'Best State: {best_state}, Minimum Cost Achieved: {min_cost * 10e-6}')


Episode: 1000, Total Reward: 15649.628732652349, Minimum Cost: 10697.951331581873
Episode: 2000, Total Reward: 12124.758100663214, Minimum Cost: 10250.359715178585
Episode: 3000, Total Reward: 15332.151681270785, Minimum Cost: 10250.359715178585
Episode: 4000, Total Reward: 19206.428868663243, Minimum Cost: 9937.462957810445
Episode: 5000, Total Reward: 15003.952710084448, Minimum Cost: 9854.819294708748
Episode: 6000, Total Reward: 13524.45899893501, Minimum Cost: 9854.819294708748
Episode: 7000, Total Reward: 13596.489891813279, Minimum Cost: 9854.819294708748


KeyboardInterrupt: 